In [ ]:
# read T5 scores for each seg
import collections
import math
from tqdm import tqdm
scores = collections.defaultdict(dict)
# Merge all passage scores into a list of scores
with open('./d2qseg.dev.tsv') as f_query_run_ids, open(
        './mono_preds_d2qseg.dev.t5') as f_pred:
    for line_query_passage_id, line_pred in tqdm(zip(f_query_run_ids, f_pred)):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        _, score = line_pred.strip().split('\t')
        score = float(score)
        scores[query_id][docseg_id] = score

20873998it [00:48, 366843.94it/s]

In [ ]:
#get T5 for each seg for bm25+d2q top1000 and take maxP for each doc and reranked 
examples = collections.defaultdict(dict)
with open('./run.doc.dev.1000.small.tsv') as fcan, open('./run.monoT5.doc.1000.trec', 'w') as fout:
    for line_query_passage_id in tqdm(fcan):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        doc_id = docseg_id.split('#')[0]
        score = scores[query_id][docseg_id]
        score = float(score)
        if doc_id not in examples[query_id]:
            examples[query_id][doc_id] = [score]
        else:
            examples[query_id][doc_id].append(score)
    
    for query_id, doc_ids_scores in tqdm(examples.items()):
        doc_ids_scores = [
            (doc_id, max(scores))
            for doc_id, scores in doc_ids_scores.items()]

        doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
        rank = 1
        for doc_id, score in doc_ids_scores:
            score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
            score += 1e-6 / rank
            fout.write(f'{query_id} Q0 {doc_id} {rank} {score} T5\n')
            rank += 1

5191985it [00:13, 394191.20it/s]
100%|██████████| 5193/5193 [00:05<00:00, 865.89it/s]


In [ ]:
#only takes top 100 for each doc for msmarco_doc_eval.py
with open ('./run.monoT5.doc.1000.tsv', 'w') as fout:
    for query_id, doc_ids_scores in tqdm(examples.items()):
            doc_ids_scores = [
                (doc_id, max(scores))
                for doc_id, scores in doc_ids_scores.items()]

            doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
            rank = 1
            for doc_id, score in doc_ids_scores:
                if (rank < 100):
                    score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
                    score += 1e-6 / rank
                    fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
                rank += 1

100%|██████████| 5193/5193 [00:02<00:00, 2455.47it/s]


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
 --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.1000.tsv

Quantity of Documents ranked for each query is as expected. Evaluating
#####################
MRR @100: 0.40878809930075677
QueriesRanked: 5193
#####################


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.monoT5.doc.1000.tsv --output ./run.monoT5.doc.1000.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt ./run.monoT5.doc.1000.trec

map                   	all	0.4088
ndcg_cut_10           	all	0.4760
